# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
from secrets_1 import IEX_CLOUD_API_TOKEN
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2339741365652,
 'week52high': 146.41,
 'week52low': 67.85,
 'week52highSplitAdjustOnly': 148.18,
 'week52lowSplitAdjustOnly': 69.5,
 'week52change': 0.925933941125929,
 'sharesOutstanding': 17277526006,
 'float': 0,
 'avg10Volume': 95068067,
 'avg30Volume': 96851625,
 'day200MovingAvg': 128.49,
 'day50MovingAvg': 124.69,
 'employees': 151248,
 'ttmEPS': 3.73,
 'ttmDividendRate': 0.8434162347545826,
 'dividendYield': 0.006290038778124111,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-22',
 'nextEarningsDate': '2021-04-26',
 'peRatio': 36.79861955659427,
 'beta': 1.333029662013423,
 'maxChangePercent': 51.36849922085766,
 'year5ChangePercent': 4.443259500395371,
 'year2ChangePercent': 1.792266608306878,
 'year1ChangePercent': 0.9265567070622729,
 'ytdChangePercent': 0.013106135829431525,
 'month6ChangePercent': 0.1369788433318837,
 'month3ChangePercent': 0.0576220176939752,
 'month1ChangePercent': 0.06953297504904382,
 'day30ChangePercen

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.9265567070622729

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns),
                ignore_index = True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,134.09,0.746133,N/A
1,AAL,21.74,0.994189,N/A
2,AAP,205.82,0.677141,N/A
3,AAPL,138.20,0.91759,N/A
4,ABBV,111.10,0.394964,N/A
...,...,...,...,...
500,YUM,119.59,0.509885,N/A
501,ZBH,180.95,0.587974,N/A
502,ZBRA,510.04,1.58227,N/A
503,ZION,56.32,1.21064,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.01,4.55909,N/A
1,179,FCX,36.80,4.11729,N/A
2,208,GPS,33.71,3.40488,N/A
3,78,CARR,44.82,2.53874,N/A
4,272,KSS,59.52,2.46389,N/A
5,24,ALGN,616.35,2.44197,N/A
6,462,URI,328.52,2.43742,N/A
7,129,DFS,98.90,2.35382,N/A
8,349,NWSA,28.52,2.30329,N/A
9,410,SIVB,532.00,2.29934,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [42]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio")
    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a numbe! \n")
        print("Please try again \n")
        portfolio_size = input("Enter the size of your portfolio")
        float(portfolio_size)

portfolio_input()
print(portfolio_size)

1000000


In [43]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])

final_dataframe

C:\Users\ayush\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.01,4.55909,294
1,179,FCX,36.80,4.11729,543
2,208,GPS,33.71,3.40488,593
3,78,CARR,44.82,2.53874,446
4,272,KSS,59.52,2.46389,336
5,24,ALGN,616.35,2.44197,32
6,462,URI,328.52,2.43742,60
7,129,DFS,98.90,2.35382,202
8,349,NWSA,28.52,2.30329,701
9,410,SIVB,532.00,2.29934,37


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [72]:
hqm_columns = [
                'Ticker',
                'Price',
                'Number of Shares to Buy',
                'One-Year Price Return',
                'One-Year Price Return Percentile',
                'Six-Month Price Return',
                'Six-Month Price Return Percentile',
                'Three-Month Price Return',
                'Three-Month Price Return Percentile',
                'One-Month Price Return',
                'One-Month Price Return Percentile',
                'HQM Score'
            ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns),
                ignore_index = True
        )

hqm_dataframe['One-Year Price Return'].fillna(0.0,inplace = True)
hqm_dataframe['Six-Month Price Return'].fillna(0.0,inplace = True)
hqm_dataframe['Three-Month Price Return'].fillna(0.0,inplace = True)
hqm_dataframe['One-Month Price Return'].fillna(0.0,inplace = True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentile,Six-Month Price Return,Six-Month Price Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile,One-Month Price Return,One-Month Price Return Percentile,HQM Score
0,A,134.59,N/A,0.739103,N/A,0.277386,N/A,0.071107,N/A,0.098650,N/A,N/A
1,AAL,21.73,N/A,1.004768,N/A,0.796316,N/A,0.410905,N/A,-0.103902,N/A,N/A
2,AAP,208.38,N/A,0.675997,N/A,0.260556,N/A,0.171890,N/A,0.081199,N/A,N/A
3,AAPL,135.70,N/A,0.888557,N/A,0.136807,N/A,0.057121,N/A,0.069153,N/A,N/A
4,ABBV,113.70,N/A,0.385785,N/A,0.293147,N/A,-0.011965,N/A,-0.015075,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.46,N/A,0.517186,N/A,0.213476,N/A,0.114959,N/A,0.088932,N/A,N/A
501,ZBH,178.60,N/A,0.600718,N/A,0.177220,N/A,0.078131,N/A,0.076150,N/A,N/A
502,ZBRA,498.71,N/A,1.607890,N/A,0.753544,N/A,0.258812,N/A,0.083892,N/A,N/A
503,ZION,54.98,N/A,1.168563,N/A,0.845360,N/A,0.154918,N/A,-0.005009,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [73]:
from scipy.stats import percentileofscore as score
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Price Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentile,Six-Month Price Return,Six-Month Price Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile,One-Month Price Return,One-Month Price Return Percentile,HQM Score
0,A,134.59,N/A,0.739103,0.590099,0.277386,0.534653,0.071107,0.29703,0.098650,0.827723,N/A
1,AAL,21.73,N/A,1.004768,0.760396,0.796316,0.928713,0.410905,0.988119,-0.103902,0.0257426,N/A
2,AAP,208.38,N/A,0.675997,0.544554,0.260556,0.50099,0.171890,0.691089,0.081199,0.718812,N/A
3,AAPL,135.70,N/A,0.888557,0.69505,0.136807,0.30099,0.057121,0.253465,0.069153,0.659406,N/A
4,ABBV,113.70,N/A,0.385785,0.29505,0.293147,0.552475,-0.011965,0.0732673,-0.015075,0.116832,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.46,N/A,0.517186,0.427723,0.213476,0.427723,0.114959,0.453465,0.088932,0.784158,N/A
501,ZBH,178.60,N/A,0.600718,0.50495,0.177220,0.374257,0.078131,0.326733,0.076150,0.69901,N/A
502,ZBRA,498.71,N/A,1.607890,0.924752,0.753544,0.906931,0.258812,0.881188,0.083892,0.738614,N/A
503,ZION,54.98,N/A,1.168563,0.815842,0.845360,0.946535,0.154918,0.641584,-0.005009,0.154455,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [74]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Price Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentile,Six-Month Price Return,Six-Month Price Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile,One-Month Price Return,One-Month Price Return Percentile,HQM Score
0,A,134.59,N/A,0.739103,0.590099,0.277386,0.534653,0.071107,0.29703,0.098650,0.827723,0.562376
1,AAL,21.73,N/A,1.004768,0.760396,0.796316,0.928713,0.410905,0.988119,-0.103902,0.0257426,0.675743
2,AAP,208.38,N/A,0.675997,0.544554,0.260556,0.50099,0.171890,0.691089,0.081199,0.718812,0.613861
3,AAPL,135.70,N/A,0.888557,0.69505,0.136807,0.30099,0.057121,0.253465,0.069153,0.659406,0.477228
4,ABBV,113.70,N/A,0.385785,0.29505,0.293147,0.552475,-0.011965,0.0732673,-0.015075,0.116832,0.259406
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,122.46,N/A,0.517186,0.427723,0.213476,0.427723,0.114959,0.453465,0.088932,0.784158,0.523267
501,ZBH,178.60,N/A,0.600718,0.50495,0.177220,0.374257,0.078131,0.326733,0.076150,0.69901,0.476238
502,ZBRA,498.71,N/A,1.607890,0.924752,0.753544,0.906931,0.258812,0.881188,0.083892,0.738614,0.862871
503,ZION,54.98,N/A,1.168563,0.815842,0.845360,0.946535,0.154918,0.641584,-0.005009,0.154455,0.639604


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [75]:
hqm_dataframe.sort_values('HQM Score',ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True,inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentile,Six-Month Price Return,Six-Month Price Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile,One-Month Price Return,One-Month Price Return Percentile,HQM Score
0,LB,67.40,N/A,4.638966,1,1.043083,0.982178,0.494131,0.99604,0.111816,0.891089,0.967327
1,MHK,204.40,N/A,1.744204,0.940594,0.956051,0.974257,0.378526,0.974257,0.121956,0.920792,0.952475
2,AMAT,137.89,N/A,1.595814,0.922772,1.168428,0.984158,0.300399,0.928713,0.132496,0.936634,0.943069
3,NUE,83.42,N/A,1.314454,0.869307,0.661163,0.855446,0.459229,0.994059,0.162127,0.976238,0.923762
4,FCX,36.40,N/A,4.084866,0.99802,1.252733,0.986139,0.270861,0.893069,0.097003,0.815842,0.923267
5,HPQ,34.30,N/A,1.308988,0.865347,0.762556,0.910891,0.366364,0.962376,0.119368,0.908911,0.911881
6,LRCX,656.90,N/A,1.464846,0.89901,0.797110,0.930693,0.182963,0.734653,0.184803,0.990099,0.888614
7,HIG,70.30,N/A,0.847988,0.665347,0.804423,0.934653,0.299370,0.924752,0.171816,0.984158,0.877228
8,PWR,98.14,N/A,1.982522,0.962376,0.567290,0.807921,0.262229,0.887129,0.098281,0.819802,0.869307
9,LNC,65.90,N/A,1.443333,0.891089,0.996008,0.980198,0.278939,0.90495,0.072428,0.683168,0.864851


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [76]:
portfolio_input()


In [77]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])

hqm_dataframe

C:\Users\ayush\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Price Return Percentile,Six-Month Price Return,Six-Month Price Return Percentile,Three-Month Price Return,Three-Month Price Return Percentile,One-Month Price Return,One-Month Price Return Percentile,HQM Score
0,LB,67.40,296,4.638966,1,1.043083,0.982178,0.494131,0.99604,0.111816,0.891089,0.967327
1,MHK,204.40,97,1.744204,0.940594,0.956051,0.974257,0.378526,0.974257,0.121956,0.920792,0.952475
2,AMAT,137.89,145,1.595814,0.922772,1.168428,0.984158,0.300399,0.928713,0.132496,0.936634,0.943069
3,NUE,83.42,239,1.314454,0.869307,0.661163,0.855446,0.459229,0.994059,0.162127,0.976238,0.923762
4,FCX,36.40,549,4.084866,0.99802,1.252733,0.986139,0.270861,0.893069,0.097003,0.815842,0.923267
5,HPQ,34.30,583,1.308988,0.865347,0.762556,0.910891,0.366364,0.962376,0.119368,0.908911,0.911881
6,LRCX,656.90,30,1.464846,0.89901,0.797110,0.930693,0.182963,0.734653,0.184803,0.990099,0.888614
7,HIG,70.30,284,0.847988,0.665347,0.804423,0.934653,0.299370,0.924752,0.171816,0.984158,0.877228
8,PWR,98.14,203,1.982522,0.962376,0.567290,0.807921,0.262229,0.887129,0.098281,0.819802,0.869307
9,LNC,65.90,303,1.443333,0.891089,0.996008,0.980198,0.278939,0.90495,0.072428,0.683168,0.864851


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [83]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [84]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [85]:
column_formats = {
                'A' : ['Ticker', string_template],
                'B' : ['Price', dollar_template],
                'C' : ['Number of Shares to Buy', integer_template],
                'D' : ['One-Year Price Return', percent_template],
                'E' : ['One-Year Return Percentile', percent_template],
                'F' : ['Six-Month Price Return', percent_template],
                'G' : ['Six-Month Price Return Percentile', percent_template],
                'H' : ['Three-Month Price Return', percent_template],
                'I' : ['Three-Month Price Return Percentile', percent_template],
                'J' : ['One-Month Price Return', percent_template],
                'K' : ['One-Month Price Return Percentile', percent_template],
                'L' : ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()


## Saving Our Excel Output

As before, saving our Excel output is very easy: